# Module Import

In [ ]:
from chemdataextractor import Document
from chemdataextractor.reader import PdfReader
from chemdataextractor.parse import R, I, join, AutoSentenceParser, AutoTableParser
from chemdataextractor.model.units import TemperatureModel
from chemdataextractor.model import StringType, ModelType, Compound

import os
import pandas as pd

# Property Model

In [ ]:
class MeltingPoint(TemperatureModel):
    """ A melting point measurement """
    
    specifier_expression = (
        R("[Mm]elting") + R("point(s?)")
        | R("[Mm]elting") + R("temperature(s?)")
        | I("Tm")
        | I("mp")
        | I("m.p")
        | I("m.p.")
        | I("MP")
        | I("M.P")
        | I("M.P.")
    ).add_action(join)
    
    specifier = StringType(parse_expression=specifier_expression, required=True)
    compound = ModelType(Compound, contextual=True, required=True)
    parsers = [AutoSentenceParser(),  AutoTableParser()]

# Auto Extraction

In [ ]:
folder = 'ACS_Organometallics_Article' # Targeted journal in this study, https://pubs.acs.org/journal/orgnd7
file_list = os.listdir(folder)

In [ ]:
records = []

for i in range(len(file_list)):
    article = file_list[i]
    print(article)
    
    try:
        f = open(folder + "/" + article, 'rb')
        doc = Document.from_file(f, readers=[PdfReader()])

        extraction_model = MeltingPoint
        doc.models = [extraction_model]
        
        parsed_information = doc.records.serialize()

        for j in parsed_information:
            if "Compound" in j:
                continue
            j["Article name"] = article
            j["Extraction Model"] = extraction_model
            records.append(j)
            
    except:
        print('Unable to read document')

In [ ]:
Auto_Extraction_Records = pd.DataFrame(records)

Auto_Extraction_Records.to_csv("ACS_OMs.csv", encoding='utf-8-sig')